##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 자동미분(Automatic differentiation) 과 그래디언트 테이프

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/eager/automatic_differentiation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/eager/automatic_differentiation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/eager/automatic_differentiation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

이전 튜토리얼에서 우리는 Tensor와 Tensor의 연산들에 대해서 알아보았습니다. 이번 튜토리얼에서는 머신러닝 모델을 최적화할 수 있는 주요 기술인 [자동미분(automatic differentiation)](https://en.wikipedia.org/wiki/Automatic_differentiation)에 대해 알아보겠습니다.

## 설정


In [ ]:
import tensorflow as tf

tf.enable_eager_execution()

## 그래디언트 테이프(Gradient Tape)

텐서플로우는 자동미분(주어진 입력 변수에 따른 기울기 연산)을 위한 [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) API를 제공합니다. `tf.GradientTape`는 안에서 실행된 모든 연산을 tape에 "기록"합니다. 그리고 [역방향 미분(reverse mode differentiation)](https://en.wikipedia.org/wiki/Automatic_differentiation)을 사용하여 기록된 연산의 그래디언트를 계산하기 위해 각각의 기록된 연산들과 관련된 테이프와 그래디언트들을 사용합니다. 

예를 들면:

In [ ]:
x = tf.ones((2, 2))
  
with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# 입력 tensor x에 관한 z의 도함수
dz_dx = t.gradient(z, x)
for i in [0, 1]:
  for j in [0, 1]:
    assert dz_dx[i][j].numpy() == 8.0

또한 `tf.GradientTape` 컨텍스트에 기록되는 동안 계산된 중간 출력값의 그래디언트를 계산할 수 있습니다.

In [ ]:
x = tf.ones((2, 2))
  
with tf.GradientTape() as t:
  t.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# 중간값 y에 관한 z의 도함수 계산을 위한 테이프 사용
# intermediate value y.
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0

초기값으로 GradientTape.gradient() 메소드가 호출되면 GradientTape에 포함된 리소스가 해체되게 설정돼있습니다. 동일한 연산을 통해서 여러 그래디언트를 계산하려면, `지속성있는(persistent)` 그래디언트 테이프를 생성하면 됩니다. `persistent`는 `gradient()` 메소드의 다중 호출을 허용합니다. 테이프 객체가 쓰레기 수집(garbage collection)될때 리소스는 해체됩니다.

In [ ]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
  y = x * x
  z = y * y
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
dy_dx = t.gradient(y, x)  # 6.0
del t  # 리소스가 해체됩니다.

### 제어흐름(Control Flow) 기록

테이프가 실행되는데로 연산을 기록하기 때문에, 파이썬 제어흐름(예를 들어 `if` `while`, `for`문 같은)은 자연스럽게 처리됩니다. 

In [ ]:
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output = tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
  return t.gradient(out, x) 

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0


### 고차원(Higher-order) 그래디언트

`GradientTape` 컨텍스트 매니저안에 있는 연산들은 자동미분을 위해 기록됩니다. 만약 그래디언트가 컨텍스트 안에서 연산되면 그래디언트 연산 또한 기록되어집니다. 그 결과 똑같은 API가 고차원 그래디언트에서도 잘 작동합니다. 예를 들면:

In [ ]:
x = tf.Variable(1.0)  # 1.0으로 초기화된 텐서플로우 변수 생성

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x
  # t 컨텍스트 매니저 안의 그래디언트 연산
  # 이것은 또한 그래디언트 계산이 미분가능하다는것을 의미합니다. 
  dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

## 다음 단계

이번 튜토리얼에서는 텐서플로우에서 그래디언트 계산법을 배웠습니다. 이를 통해 우리는 신경망을 구축하고 훈련시키는 데 필요한 기본 요소를 충분히 확보 할 수 있습니다.